In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [3]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


In [4]:
import xml.etree.ElementTree as xet

In [5]:
filename = df['filepath'][0]

In [6]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text 
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [7]:
getFilename(filename)

'./images\\N1.jpeg'

In [9]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['./images\\N1.jpeg',
 './images\\N100.jpeg',
 './images\\N101.jpeg',
 './images\\N102.jpeg',
 './images\\N103.jpeg',
 './images\\N104.jpeg',
 './images\\N105.jpeg',
 './images\\N106.jpeg',
 './images\\N107.jpeg',
 './images\\N108.jpeg',
 './images\\N109.jpeg',
 './images\\N11.jpeg',
 './images\\N110.jpeg',
 './images\\N111.jpeg',
 './images\\N112.jpeg',
 './images\\N113.jpeg',
 './images\\N114.jpeg',
 './images\\N116.jpeg',
 './images\\N117.jpeg',
 './images\\N118.jpeg',
 './images\\N119.jpeg',
 './images\\N12.jpeg',
 './images\\N120.jpeg',
 './images\\N121.jpeg',
 './images\\N122.jpeg',
 './images\\N123.jpeg',
 './images\\N124.jpeg',
 './images\\N126.jpeg',
 './images\\N127.jpeg',
 './images\\N128.jpeg',
 './images\\N129.jpeg',
 './images\\N130.jpeg',
 './images\\N131.jpeg',
 './images\\N132.jpeg',
 './images\\N133.jpeg',
 './images\\N134.jpeg',
 './images\\N135.jpeg',
 './images\\N136.jpeg',
 './images\\N137.jpeg',
 './images\\N138.jpeg',
 './images\\N139.jpeg',
 './images\\N14.jpeg

In [10]:
file_path = image_path[0]
file_path

'./images\\N1.jpeg'

In [ ]:
img = cv2.imread(file_path)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
cv2.rectangle(img,(1093,645),(1396,727),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Data Preprocessing


In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [20]:
labels = df.iloc[:,1:].values

In [25]:
data=[]
output=[]
for ind in range(len(image_path)) :
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) 
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [30]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [31]:
X.shape,y.shape

((225, 224, 224, 3), (225, 4))

In [40]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state = 0)


Deep Learning


In [34]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [36]:
inception_resnet = InceptionV3(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(64,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [37]:
model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6))
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_94 (Conv2D)            │ (None, 111, 111, 32)      │             864 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_94        │ (None, 111, 111, 32)      │              96 │ conv2d_94[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_94 (Activation)    │ (None, 111, 111, 32)      │               0 │ batch_normalization_94[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_95 (Conv2D)            │ (None, 109, 109, 32)      │           9,216 │ activation_94[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_95        │ (None, 109, 109, 32)      │              96 │ conv2d_95[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_95 (Activation)    │ (None, 109, 109, 32)      │               0 │ batch_normalization_95[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_96 (Conv2D)            │ (None, 109, 109, 64)      │          18,432 │ activation_95[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_96        │ (None, 109, 109, 64)      │             192 │ conv2d_96[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_96 (Activation)    │ (None, 109, 109, 64)      │               0 │ batch_normalization_96[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_4               │ (None, 54, 54, 64)        │               0 │ activation_96[0][0]        │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_97 (Conv2D)            │ (None, 54, 54, 80)        │           5,120 │ max_pooling2d_4[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_97        │ (None, 54, 54, 80)        │             240 │ conv2d_97[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_97 (Activation)    │ (None, 54, 54, 80)        │               0 │ batch_normalization_97[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_98 (Conv2D)            │ (None, 52, 52, 192)       │         138,24

 Total params: 25,079,908 (95.67 MB)

 Trainable params: 3,277,124 (12.50 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

### Object Detection Model Training

In [38]:
from tensorflow.keras.callbacks import TensorBoard
tfb = TensorBoard('object_detection')

In [41]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 257ms/step - loss: 0.0837 - val_loss: 0.0548
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 159ms/step - loss: 0.0569 - val_loss: 0.0468
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 158ms/step - loss: 0.0399 - val_loss: 0.0445
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - loss: 0.0442 - val_loss: 0.0433
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 167ms/step - loss: 0.0376 - val_loss: 0.0417
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 173ms/step - loss: 0.0322 - val_loss: 0.0398
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 172ms/step - loss: 0.0287 - val_loss: 0.0386
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 189ms/step - loss: 0.0295 - val_loss: 0.0378
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - loss: 0.0267 - val_loss: 0.0366
Epoch 10/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 169ms/step - loss: 0.0256 - val_loss: 0.0357
Epoch 11/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 179ms/step - loss: 0.0224 - val_loss: 0.0352
Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3

18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 185ms/step - loss: 0.0013 - val_loss: 0.0268
Epoch 69/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step - loss: 0.0015 - val_loss: 0.0268
Epoch 70/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 193ms/step - loss: 9.2455e-04 - val_loss: 0.0268
Epoch 71/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 185ms/step - loss: 0.0012 - val_loss: 0.0268
Epoch 72/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 186ms/step - loss: 0.0011 - val_loss: 0.0268
Epoch 73/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step - loss: 0.0011 - val_loss: 0.0268
Epoch 74/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 184ms/step - loss: 0.0011 - val_loss: 0.0268
Epoch 75/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 194ms/step - loss: 9.1425e-04 - val_loss: 0.0268
Epoch 76/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step - loss: 7.3541e-04 - val_loss: 0.0268
Epoch 77/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 186ms/step - loss: 6.7745e-04 - val_loss: 0.0268
Epoch 78/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 183ms/step - loss: 8.3446e-04 - val_loss: 0.0268
Epoch 79/100
18/18 ━━━━━━

In [43]:
model.save('./models/object_detection.h5')